In [99]:
import random
import requests
from datetime import datetime
import sqlite3
# import pandas as pd
import json
import hashlib
import string

In [100]:
def gerar_str_aleatoria(tamanho=10):
    caracteres = string.ascii_letters + string.digits  # Letras maiúsculas, minúsculas e números
    return ''.join(random.choices(caracteres, k=tamanho))

In [121]:
def calcular_tempo_decorrido(inicio, fim):
    delta = fim - inicio
    segundos = delta.total_seconds()

    if segundos < 60:
        return f"{segundos:.2f} segundos"
    elif segundos < 3600:
        minutos = segundos / 60
        return f"{minutos:.2f} minutos"
    elif segundos < 86400:
        horas = segundos / 3600
        return f"{horas:.2f} horas"
    else:
        dias = segundos / 86400
        return f"{dias:.2f} dias"

In [6]:
def conectar_banco(nome_banco=fr"E:\Projetos\Jupyter\lotofacil.db"):
    """Estabelece conexão com o banco de dados SQLite3."""
    conexao = sqlite3.connect(nome_banco)
    cursor = conexao.cursor()
    return conexao, cursor

In [5]:
def select(query):
    conexao, cursor = conectar_banco()        
    
    # Criar o DataFrame a partir da consulta SQL
    # df = pd.read_sql_query(query, conexao)
    
    # Executar a consulta SQL
    cursor.execute(query)
    
    # Obter os nomes das colunas
    colunas = [descricao[0] for descricao in cursor.description]
    
    # Obter todos os resultados da consulta
    dados = cursor.fetchall()
    
    # Transformar os dados em uma lista de dicionários (JSON-friendly)
    resultado_json = [dict(zip(colunas, linha)) for linha in dados]        
    
    # Converter para JSON
    json_saida = json.dumps(resultado_json, indent=4, ensure_ascii=False)                
    
    conexao.close()
    
    return json.loads(json_saida)

In [4]:
def criar_aposta():
    return sorted(random.sample(range(1, 26), 15))



In [3]:
def populacao_inicial(tamanho):
    return [criar_aposta() for _ in range(tamanho)]



In [8]:
def getNumerosFrequentes():
    query = f"""
        WITH numeros AS (
        SELECT dezena_1 AS numero FROM dataframe
        UNION ALL
        SELECT dezena_2 FROM dataframe
        UNION ALL
        SELECT dezena_3 FROM dataframe
        UNION ALL
        SELECT dezena_4 FROM dataframe
        UNION ALL
        SELECT dezena_5 FROM dataframe
        UNION ALL
        SELECT dezena_6 FROM dataframe
        UNION ALL
        SELECT dezena_7 FROM dataframe
        UNION ALL
        SELECT dezena_8 FROM dataframe
        UNION ALL
        SELECT dezena_9 FROM dataframe
        UNION ALL
        SELECT dezena_10 FROM dataframe
        UNION ALL
        SELECT dezena_11 FROM dataframe
        UNION ALL
        SELECT dezena_12 FROM dataframe
        UNION ALL
        SELECT dezena_13 FROM dataframe
        UNION ALL
        SELECT dezena_14 FROM dataframe
        UNION ALL
        SELECT dezena_15 FROM dataframe
    )
    SELECT numero, COUNT(*) AS quantidade
    FROM numeros
    GROUP BY numero
    ORDER BY quantidade DESC
    LIMIT 15;
    """
    
    dados = select(query)
    
    numeros_frequentes = [dado['numero'] for dado in dados]   
    
    return numeros_frequentes         
    

In [78]:
def fitness(individuo):    
    
    pontuacao_maxima = 100
    pontos_acertos = 0    
    
    # Pontos acertos
    if individuo['quinze_acertos'] >= 1:
        pontos_acertos += individuo['quinze_acertos'] * 300
    if individuo['quatorze_acertos'] >= 1:
        pontos_acertos += individuo['quatorze_acertos'] * 150
    if individuo['treze_acertos'] >= 1:
        pontos_acertos += individuo['treze_acertos'] * 50
    if individuo['doze_acertos'] >= 1:
        pontos_acertos += individuo['doze_acertos'] * 10
    if individuo['onze_acertos'] >= 1:
        pontos_acertos += individuo['onze_acertos'] * 2
    
    # Pontos simples por estarem dentro dos números mais frequentes
    numeros_frequentes = getNumerosFrequentes()        
    pontos_frequencia = sum(1 for num in individuo['cromossomo'] if num in numeros_frequentes)
    
    # Pontuação por balanceamento
    pares = 0    
    for gene in individuo['cromossomo']:
        if gene % 2 == 0:
            pares += 1        
    
    pontos_balanceamento = 0
    if pares >= 7 and pares <= 8:
        pontos_balanceamento = 15
    elif pares == 6 or pares == 9:
        pontos_balanceamento = 10
    elif pares == 5 or pares == 10:
        pontos_balanceamento = 5
           
        
    total = pontos_acertos + pontos_frequencia + pontos_balanceamento
    total /= pontuacao_maxima
    
    # print(f'{pontos_acertos} Acertos + {pontos_frequencia} de frequencia + {pontos_balanceamento} pontos de balanceamento. \t TOTAL: {total}')
    return total            



In [70]:

def cruzamento(pai1, pai2):
    pai1 = pai1['cromossomo']
    pai2 = pai2['cromossomo']
    
    filho = list(set(pai1[:8] + pai2[8:]))        
    
    while len(filho) < 15:        
        num = random.randint(1, 25)
        if num not in filho:
            filho.append(num)
    
    ordenado = sorted(filho)            
    
    item = {
        'cromossomo': ordenado,
        'fitness': 0,
        'onze_acertos': 0,
        'doze_acertos': 0,
        'treze_acertos': 0,
        'quatorze_acertos': 0,
        'quinze_acertos': 0
    }
    
    item['fitness'] = fitness(item)
    
    return item



In [71]:
def mutacao(filho, taxa=0.2):
    
    aposta = filho['cromossomo']    
    
    if random.random() < taxa:        
        idx = random.randint(0, 14)
        novo_num = random.randint(1, 25)
        while novo_num in aposta:
            novo_num = random.randint(1, 25)
        aposta[idx] = novo_num        
    
    item = {
        'cromossomo': aposta,
        'fitness': 0,
        'onze_acertos': 0,
        'doze_acertos': 0,
        'treze_acertos': 0,
        'quatorze_acertos': 0,
        'quinze_acertos': 0
    }
    
    item['fitness'] = fitness(item)
    
    return item



In [72]:
def selecao_inicial(populacao):
    
    dados = []
    
    # Pontuando ele
    for individuo in populacao:
        item = {
            'cromossomo': individuo,
            'fitness': 0,
            'onze_acertos': 0,
            'doze_acertos': 0,
            'treze_acertos': 0,
            'quatorze_acertos': 0,
            'quinze_acertos': 0
        }
        
        item['fitness'] = fitness(item)
        
        dados.append(item)
    
    # Ordenando ele
    dados_ordenado = sorted(dados, key=lambda x: x['fitness'], reverse=True)    
    
    return dados_ordenado
        
    

In [73]:
def selecao(populacao):        
       
    # Aqui já rodou uma vez e têm-se os dados de acertos
    for individuo in populacao:
        individuo['fitness'] = fitness(individuo)       
       
    # Ordenando ele
    dados_ordenado = sorted(populacao, key=lambda x: x['fitness'], reverse=True)    
    
    return dados_ordenado
        
    

In [14]:
def getNumeroAcertos(jogo, individuo):
    
    dezenas_real = [
        jogo['dezena_1'], jogo['dezena_2'], jogo['dezena_3'], jogo['dezena_4'], jogo['dezena_5'], 
        jogo['dezena_6'], jogo['dezena_7'], jogo['dezena_8'], jogo['dezena_9'], jogo['dezena_10'], 
        jogo['dezena_11'], jogo['dezena_12'], jogo['dezena_13'], jogo['dezena_14'], jogo['dezena_15'] 
    ]
    
    dezenas_test = individuo['cromossomo']
    
    # Encontrando a interseção
    comuns = set(dezenas_real) & set(dezenas_test)

    # Contando os números em comum
    quantidade = len(comuns)

    # print(f"Números em comum: {comuns}")
    # print(f"Quantidade: {quantidade}")
    
    return quantidade

In [115]:
def saveResultado(populacao, nome, geracao, loteria='lotofacil'):
    
    conexao, cursor = conectar_banco()       
    
    for individuo in populacao:        
        
        pares = 0    
        for gene in individuo['cromossomo']:
            if gene % 2 == 0:
                pares += 1   
        
        dezena_1, dezena_2, dezena_3, dezena_4, dezena_5, dezena_6, dezena_7, dezena_8, dezena_9, dezena_10, dezena_11, dezena_12, dezena_13, dezena_14, dezena_15 = individuo['cromossomo']
        
        insert = f"""
            INSERT INTO resultado (
            nome, loteria, geracao, fitness,
            dezena_1, dezena_2, dezena_3, dezena_4, dezena_5, 
            dezena_6, dezena_7, dezena_8, dezena_9, dezena_10, 
            dezena_11, dezena_12, dezena_13, dezena_14, dezena_15, 
            quinze_acertos, quatorze_acertos, treze_acertos, doze_acertos, onze_acertos, 
            pares, impares
            ) VALUES (
                '{nome}', '{loteria}', {geracao}, {individuo['fitness']},
                {dezena_1}, {dezena_2}, {dezena_3}, {dezena_4}, {dezena_5},
                {dezena_6}, {dezena_7}, {dezena_8}, {dezena_9}, {dezena_10},
                {dezena_11}, {dezena_12}, {dezena_13}, {dezena_14}, {dezena_15},
                {individuo['quinze_acertos']}, {individuo['quatorze_acertos']}, {individuo['treze_acertos']}, {individuo['doze_acertos']}, {individuo['onze_acertos']}, 
                {pares}, {(15-pares)}
            );
        """
        
        cursor.execute(insert)
        
    conexao.commit()
    conexao.close()
        
        

In [127]:
def algoritmo_genetico(geracoes=100, tamanho_populacao=50, jogos_treinamento=100):        
    
    geracoes += 1
    populacao = populacao_inicial(tamanho_populacao)            
    passam_direto_percent = 0.1
    passam_direto = tamanho_populacao * passam_direto_percent
    if passam_direto < 1:
        passam_direto = 1        
    progress = 0
    
    the_best = []
    the_most_wanted = []
    
    jogos_base = select(f'select * from dataframe order by random() limit {jogos_treinamento}')
    
    # Log dos resultados
    save = True
    nome = hashlib.md5(gerar_str_aleatoria().encode()).hexdigest()
    
    inicio = datetime.now()    
    
    for geracao in range(geracoes):                                                                
        
        if geracao == 0:
            populacao = selecao_inicial(populacao) # Isso coloca no padrão json e ordena por fitness    
            print(f'\n\t\t{geracao+1}° Geracao \t [Calculando...]\n')                    
        else:
            populacao = selecao(populacao) # Aplica o fitness com os acertos passados
            
            # Salvando os melhores dos melhores
            if len(the_best) == 0: # na primeira geração só passamos direto sem comparar
                the_best = populacao
            else:                                                                                                         
                for individuo in populacao:
                    for idx, best in enumerate(the_best):
                        
                        # Salvando os que acertaram as melhores apostas
                        if individuo['quatorze_acertos'] > 1 or individuo['quinze_acertos'] > 1:
                            the_most_wanted.append(individuo)
                        
                        # Salvando/Trocando na lista dos melhores
                        if individuo['fitness'] > best['fitness']:
                            the_best[idx] = individuo
                            break
                        
                if save:
                    saveResultado(populacao, nome, (geracao+1))  
                    
                fim = datetime.now()
                delta = calcular_tempo_decorrido(inicio, fim)                
                print(f'\n\t\t{geracao+1}° Geracao \t [{delta}]\n')
                inicio = datetime.now()
            
        
        # Criando a nova população
        nova_populacao = []
        # Passam direto os X primeiros colocados        
        passaram_direto = populacao[:int(passam_direto)]        
        nova_populacao = passaram_direto
        
        while len(nova_populacao) < tamanho_populacao:                        
                        
            pai1, pai2 = random.sample(populacao, 2)            
            filho = cruzamento(pai1, pai2)   
            filho = mutacao(filho)                                                    
            
            nova_populacao.append(filho)
        populacao = nova_populacao                    

        # Re-avaliar a nova população        
        for individuo in populacao:                        
            
            houve_acertos = False
            onze_acertos = 0
            doze_acertos = 0
            treze_acertos = 0
            quatorze_acertos = 0
            quinze_acertos = 0
            
            # Joga elas na selva
            for jogo in jogos_base:
                
                progress += 1
                acertos = getNumeroAcertos(jogo, individuo)
                
                if acertos == 11:
                    onze_acertos += 1
                    houve_acertos = True
                elif acertos == 12:
                    doze_acertos += 1
                    houve_acertos = True
                elif acertos == 13:
                    treze_acertos += 1
                    houve_acertos = True
                elif acertos == 14:
                    quatorze_acertos += 1
                    houve_acertos = True
                elif acertos == 15:
                    quinze_acertos += 1      
                    houve_acertos = True
                    
            # Salva o resultado no indivíduo
            individuo['onze_acertos'] = onze_acertos
            individuo['doze_acertos'] = doze_acertos
            individuo['treze_acertos'] = treze_acertos
            individuo['quatorze_acertos'] = quatorze_acertos
            individuo['quinze_acertos'] = quinze_acertos
            
            # if houve_acertos:
            #     print(f'Resultado: {individuo}')                        
            # print('-----------------------------------------')                                                                                    

    
    # Apresentando os resultados
    # print('\t\t\t\n\nTHE MOST WANTED\n\n')
    # print(the_most_wanted)
    print(f'DONE - {nome}')

In [130]:
algoritmo_genetico(geracoes=120, tamanho_populacao=1000, jogos_treinamento=3300)



		1° Geracao 	 [Calculando...]


		3° Geracao 	 [2.11 minutos]


		4° Geracao 	 [55.07 segundos]


		5° Geracao 	 [54.78 segundos]


		6° Geracao 	 [54.89 segundos]


		7° Geracao 	 [54.83 segundos]


		8° Geracao 	 [54.88 segundos]


		9° Geracao 	 [54.81 segundos]


		10° Geracao 	 [54.81 segundos]


		11° Geracao 	 [54.71 segundos]


		12° Geracao 	 [54.79 segundos]


		13° Geracao 	 [54.81 segundos]


		14° Geracao 	 [54.71 segundos]


		15° Geracao 	 [54.88 segundos]


		16° Geracao 	 [54.81 segundos]


		17° Geracao 	 [54.80 segundos]


		18° Geracao 	 [54.77 segundos]


		19° Geracao 	 [54.80 segundos]


		20° Geracao 	 [54.86 segundos]


		21° Geracao 	 [54.73 segundos]


		22° Geracao 	 [54.72 segundos]


		23° Geracao 	 [54.78 segundos]


		24° Geracao 	 [54.77 segundos]


		25° Geracao 	 [54.80 segundos]


		26° Geracao 	 [54.75 segundos]


		27° Geracao 	 [54.75 segundos]


		28° Geracao 	 [54.82 segundos]


		29° Geracao 	 [54.90 segundos]


		30° Geracao 	 [54.84 segundo